In [26]:
import os as os
import ib_insync
import pandas as pd
import sqlite3 as sqlite3
from datetime import datetime, timedelta, date
# print(ib_insync.__all__)

In [2]:
from ib_insync import *
util.startLoop()

In [3]:
# 1.0 打開連接
ib = IB()
ib.connect('127.0.0.1', port=4002, clientId=1, timeout=5)

<IB connected to 127.0.0.1:4002 clientId=1>

In [4]:
# 2.0 帳戶資金
[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

[AccountValue(account='DU365095', tag='NetLiquidationByCurrency', value='2171451.5651', currency='BASE', modelCode='')]

In [5]:
# 3.0 期貨
date1 = date.today().strftime('%Y%m')
future1 = Future('NQ', date1, 'GLOBEX')

In [9]:
# 4.0 歷史數據
bars1 = ib.reqHistoricalData(future1, endDateTime='', durationStr='60 D', barSizeSetting='1 min', whatToShow='MIDPOINT', useRTH=True)
data1 = util.df(bars1)
data1

,date,open,high,low,close,volume,average,barCount
0,2020-10-30 21:30:00,11246.875,11257.375,11230.000,11238.500,-1,-1.0,-1
1,2020-10-30 21:31:00,11238.500,11265.625,11234.625,11260.750,-1,-1.0,-1
2,2020-10-30 21:32:00,11260.750,11281.125,11257.125,11273.000,-1,-1.0,-1
3,2020-10-30 21:33:00,11273.000,11277.500,11262.250,11265.875,-1,-1.0,-1
4,2020-10-30 21:34:00,11265.875,11268.875,11242.250,11247.875,-1,-1.0,-1
...,...,...,...,...,...,...,...,...
12835,2020-12-12 05:55:00,12372.500,12373.125,12371.125,12371.500,-1,-1.0,-1
12836,2020-12-12 05:56:00,12371.500,12373.250,12370.875,12372.875,-1,-1.0,-1
12837,2020-12-12 05:57:00,12372.875,12373.500,12372.875,12373.125,-1,-1.0,-1
12838,2020-12-12 05:58:00,12373.125,12374.375,12372.375,12373.375,-1,-1.0,-1


In [27]:
# 4.1 储存sqlite
data1['date'] = pd.to_datetime(data1['date'])
data1['date'] = data1['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
data1.drop(data1.columns.difference(['date', 'open', 'high', 'low', 'close', 'volume']), 1, inplace=True)
data2 = [tuple(x) for x in data1.values]

path_db = os.path.abspath(os.path.join('data', 'nq', 'data', 'nq.db'))
db = sqlite3.connect(path_db)
cursor = db.cursor()
cursor.executemany('REPLACE INTO nq (udate, high, low, open, close, vol) VALUES (?, ?, ?, ?, ?, ?)', data2)
db.commit()
cursor.close()
db.close()

In [7]:
# 5.0 買盤
order1 = LimitOrder('BUY', 1, 12474.00+0.25)
#trade1 = ib.placeOrder(future1, order1)
#trade1

In [8]:
# 6.0 倉位
ib.positions()

[Position(account='DU365095', contract=Future(conId=383974419, symbol='NQ', lastTradeDateOrContractMonth='20201218', multiplier='20', currency='USD', localSymbol='NQZ0', tradingClass='NQ'), position=3.0, avgCost=250322.1)]

In [9]:
# 7.0 手續費
[fill.commissionReport.commission for fill in ib.fills()]

[]

In [10]:
# 8.0 賣盤
order2 = LimitOrder('SELL', 1, 12490.25-0.25)
trade2 = ib.placeOrder(future1, order2)
trade2

Trade(contract=Future(symbol='NQ', lastTradeDateOrContractMonth='202012', exchange='GLOBEX'), order=LimitOrder(orderId=48, clientId=1, action='SELL', totalQuantity=1, lmtPrice=12490.0), orderStatus=OrderStatus(orderId=48, status='PendingSubmit', filled=0, remaining=0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2020, 12, 14, 3, 1, 15, 282857, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

In [11]:
# 9.0 盤
ib.orders()

[LimitOrder(orderId=48, clientId=1, action='SELL', totalQuantity=1, lmtPrice=12490.0)]

In [ ]:
# 10.0 交易紀錄
ib.trades()

In [13]:
# 關閉連接
ib.disconnect()